# Project: Mental health in Switzerland

In this notebook, we describe the multiple steps of our preprocessing pipeline and the work done on the twitter dataset to answer our research question.

### Overview:
0. **Data Retrival:** how we access the dataset
1. **Dataset Selection and Analysis:** a first look at the dataset, exploring potential issues
2. **Dataset Cleaning:** cleaning based on the results found in _Part 1_
3. **NLP methods:** applying NLP methods to our data to retrieve relevant data
4. **Machine Learning:** using Machine Learning to further clean our dataset
5. **Analysis:** trying to find meaning in the final dataset we retrieve
6. **Conclusion**

We quickly import the libraries to be used later:

In [1]:
import numpy as np
import pandas as pd
import pickle as pkl
import pyspark as ps
import matplotlib.pyplot as plt

#Part 1
import json
from pandas.io.json import json_normalize

#Part 2
import unicodedata

#Part 3
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

## 0. Datasets Retrieval

We did not retrieve both datasets using the same method first because they did not have the same weight, but also because they were not provided on the same platform.

Our first dataset (_**twitter-swisscom**_) was retrieved from a .zip file. Thus, we had access to the entire dataset quickly allowing us to have an overview of all the tweets when analyzing it (cf _Part 1_ below).

Our second dataset (_**Spinn3r**_), however, was retrieved from the cluster. We first tried to retrieve the whole dataset but quickly realized it would be impossible to do so (it was very heavy and took a long time to be downloaded). Thus, we only extracted the first JSON file to perform our analysis. To do this, we used the following methods:

```bash
cluster$ hadoop fs -getmerge /datasets/swiss-tweet/harvest3r_twitter_data_01-01_0.json /buffer/example.json
local$ scp -r gaspar@iccluster060.iccluster.epfl.ch:/buffer/example.json <local-path>
```

## 1. Dataset Selection & Analysis

We were provided two separate datasets containing Swiss tweets. They were formated differently and contained different fields. On top of this, while the first dataset contained data collectd over multiple years, the other only covers a span of 10 months.

The goal of our first analysis was thus to decide which dataset would be more useful but also to familiarize ourselves with the chosen dataset in order to know if we need to adapt our research questions or enrich the dataset with external information in order to perform our analysis.

After the analysis (presented below), we decided to use **dataset 2** for our project. While dataset 1 contains more precise location information (longitude and latitude), dataset 2 contains a sentiment analysis field as well as a language field. As trying to categorize the language of each tweet in the first dataset was quite expensive in terms of computation – we needed to deal with the network latency of our API requests – and a lot of preprocessing was necessary to get it to work, dataset 2 was clearly better suited for our project.

### Dataset 1 (twitter-swisscom)

The dataset comes with a *txt schema*, giving us an idea of the meaning of each column in the *tsv file* containing the tweets. We were also given a sample file to get an overview of the data, but we also optained the complete set of tweets (5GB) in a _.zip_ file.

The dataset contains the following useful columns:

- **userId** : id identifying the user.
- **createdAt** : time the tweet was posted.
- **text** : content of the tweet.
- **placeLatitude** : latitude of the tweet.
- **placeLongitude** : longitude of the tweet.
- **sourceName** : username.
- **sourceUrl** : URL of the tweet.
- **followersCount** : number of followers.
- **friendsCount** : number of mutual follows.
- **statusesCount** : number of statuses of user.

The sample dataset contains a lot of NaN values, and each column contains at least 1% or more NaN values.

The complete analysis and code can be found in the [Basic Exploration dataset 1 notebook](Basic%20Exploration%20Dataset%201.ipynb)

### Dataset 2 (from Spinn3r)

#### Metadata analysis

This dataset has an elaborate description of each field available on the [spinn3r website](http://docs.spinn3r.com/?Example#content-schema). Given the amount of data present in the cluster, we only look at one day to perform our first analysis (we will later show how we scale our operations using Spark).

Unlike the previous dataset, this dataset is given in JSON format (with nested elemtns). As we could not find how to extract all the data directly using the *read_json* function provided, we use a JSON normalizer (provided in the *Pandas* libary). 

_Note: We will later see that Spark deals better with nested JSON._

The fields found in this dataset are:

In [2]:
EXAMPLE_PATH = 'swiss-tweet/example.json'

with open(EXAMPLE_PATH) as data_file:    
    data = json.load(data_file)

twitter_df = json_normalize(data)
#rename columns for convenience
twitter_df.columns = [column.replace('_source.','') for column in twitter_df.columns]
twitter_df.columns

Index(['_id', '_index', '_score', 'author_avatar_img', 'author_gender',
       'author_link', 'author_name', 'bucket', 'canonical', 'date_found',
       'domain', 'hashcode', 'index_method', 'lang', 'links', 'main',
       'main_checksum', 'main_format', 'main_length', 'mentions', 'permalink',
       'published', 'resource', 'sentiment', 'sequence', 'sequence_range',
       'site', 'source_content_checksum', 'source_content_length',
       'source_created', 'source_date_found', 'source_description',
       'source_favicon_height', 'source_favicon_width', 'source_favorites',
       'source_followers', 'source_following', 'source_handle',
       'source_hashcode', 'source_http_status', 'source_image_height',
       'source_image_src', 'source_image_width', 'source_last_posted',
       'source_last_published', 'source_last_updated', 'source_likes',
       'source_link', 'source_location', 'source_parsed_posts',
       'source_parsed_posts_max', 'source_profiles', 'source_publisher_type',


Out of all these columns, we find the ones below to be the most useful for our analysis:
- **main**: contains the content of the tweet.
- **published**: gives the time on which the content was posted.
- **source_spam_probability**: probability of tweet being spam.
- **source_location**: location of the tweet.
- **tags**: tags associated with the tweet (provided by Spinn3r).
- **lang**: language of the tweet.
- **sentiment**: sentiment score of the tweet -POSITIVE, NEGATIVE, NEUTRAL-.
- **author_gender**: gender of the author -MALE, FEMALE, UNKNOWN-.
- **source_followers**: followers of the user who tweeted.
- **source_following**: number of people the user follows.

We consider these tags to be the most useful as:

- **Main** stands at the center of our analysis. We plan to perform NLP methods in order to identify relevant tweets and use the content as well to identify related words.
- **Published** can be used to map the tweets over the duration of the year and look for seasonal changes.
- **Source_location** can be used to look at the geographical distribution of the tweets.
- **Lang** will be used to filter out unwanted languages, which we need to do in order to perform our NLP tasks.
- **Author_gender** will be used to identify the gender and look at the differences between both genders.

In [3]:
columns = ['main', 'published', 'source_spam_probability', 'source_location', 'tags', 'lang', 'sentiment',
                   'author_gender', 'source_followers', 'source_following']
twitter_df = twitter_df[columns]

#### General Distribution

While this example isn't representative (especially given that it contains the tweets posted on January, 1st), it can still give us insights on other fields. We assume that roughly the same categories of users were active on that day, meaning we can draw conclusions on the distribution of language and gender.

In [4]:
twitter_df['lang'].value_counts()

en     7350
fr     4421
de     4174
pt     2139
es     1463
und     893
it      266
in      125
tr      124
pl      124
ar      116
ja      111
ht       84
nl       71
tl       60
et       43
da       39
sv       38
zh       30
ko       25
no       25
fi       15
lt       14
ru        9
hi        7
is        7
hu        5
sl        5
lv        4
ta        3
bg        2
el        1
th        1
vi        1
Name: lang, dtype: int64

We see that English, French and German are the most frequent languages. This is good as those are the languages we plan on using.

In [5]:
twitter_df['author_gender'].value_counts()

UNKNOWN    14424
FEMALE      4044
MALE        3327
Name: author_gender, dtype: int64

We see that most accounts do not contain information on the user's gender (meaning there is no way we can have an unbiased set). However, we have a significant number of profiles where the gender is documented meaning we could use these to look at the differences in mental distress between the genders.

In [6]:
twitter_df['sentiment'].value_counts()

NEUTRAL     20357
POSITIVE      482
NEGATIVE       63
Name: sentiment, dtype: int64

Once again, we see that the vast majority of tweets are labeled as neutral while only a very small number is labeled as negative. Given our subject will look at both the **'NEUTRAL'** and **'NEGATIVE'** tweets.

_Note: we make the assumption that there are no false positives, meaning that a tweet showing signs of mental distress will not be labeled as **'POSITIVE'**. This means we can safely exclude these tweets from further analysis._

In [7]:
twitter_df['source_spam_probability'].value_counts()

0.0    21795
Name: source_spam_probability, dtype: int64

Looking at the spam probability of the tweet set, we see that not a single tweet was labeled as spam. This makes us question the accuracy of the labeling as the set of tweets on that day most certainly contains spam. However, we will still use it as we assume the chance of having false positives to be very low.

In [8]:
twitter_df[twitter_df.lang.isin(['de', 'fr', 'en'])]['source_location'].value_counts()

Switzerland           6613
Schweiz               1977
Suisse                1762
Genève                1094
Zürich                 961
Geneva                 676
Zurich                 544
Basel                  433
Bern                   363
Lausanne               287
CH                     154
Swiss                  124
Lugano                  93
St. Gallen              88
Geneve                  67
Schaffhausen            64
Fribourg                59
Svizzera                50
Luzern                  44
Winterthur              42
Baden                   37
Chur                    26
Lenzburg                23
Interlaken              21
Waldenburg              21
Biel                    19
Genf                    19
Marly                   19
Sankt Gallen            18
Solothurn               14
                      ... 
Hinwil                   2
Bâle                     2
Frauenfeld               2
Horgen                   2
Visp                     2
Glarus                   2
L

As the location seems to be language dependent, we only care about tweets written in the 3 languages we need. We see that:
- A lot of locations only differ in language (e.g. as Switzerland and Schweiz)
- The name of the locations are not always given languages we are interested in (e.g. สวิตเซอร์แลนด์)
- A vast majority of the dataset is just located in 'Switzerland'
- As opposed to dataset 1, all tweets are located in Switzerland

In [9]:
twitter_df.count()

main                       21795
published                  21795
source_spam_probability    21795
source_location            21795
tags                        3865
lang                       21795
sentiment                  20902
author_gender              21795
source_followers           21795
source_following           21795
dtype: int64

Looking at the missing values in the dataset, we see that for most tweets, the tags are missing. This may indicate that the tags won't be usefull for our analysis, but this does not influence our research at this stage.

We also note that for some tweets, the sentiment is missing. As noted on Spinn3er, this may be due to the fact that some tweets do not contain enough linguistic information. As we will filter out such tweets, the remaining set should contain sentiment. Even if doesn't, this field is not central to our analysis, it is merely used to help us filter our dataset.

In [10]:
twitter_df[~(twitter_df.sentiment.isin(['POSITIVE', 'NEGATIVE', 'NEUTRAL']))]['lang'].value_counts()

und    893
Name: lang, dtype: int64

To be sure this won't be an issue, we quickly look at all the uncategorized tweets and see that the language is unkown as well, meaning they will all be removed anyways.

#### Potential issues

While this set of tweets is not representative, we can still use it to find potential issues we might have with the tweets' content:

In [11]:
pd.set_option('display.max_colwidth', -1)
twitter_df.sample(n=10)['main']

7002     Hello everyone, have a great Friday! Looking forward to reading your tweets :) (insight by http://commun.it )                                  
16140    anime fight between Nico and his bad luck                                                                                                      
3148     """"Tratamento da doenca do Panico, Sindrome do Panico, Ansiedades Sete Lagoas http://psicologosetelagoas.com.br  313773-9185 OH7366""""       
18324    2002 @AudemarsPiguet Royal Oak Concept Watch is one of the very first Super watches. Click http://bit.ly/1D9TojO  pic.twitter.com/aJ80gjWTj3   
1196     Feliz inicio de año! Todo lo bueno y mejor para todos!                                                                                         
5412     ouais comme toute les années ahhahah https://twitter.com/lxviss/status/682736832581337088 … J'aurai fini 2015 célibataire mdr                  
734      What if Star Wars had been realised by Fred Astaire? Perfect #alternatehi

Looking at the sample, we immediatly see that the tweets containing links are not relevant to our research question (they are mostly news or adds).

In [12]:
twitter_df.main[twitter_df.main.map(lambda x: 'http://' in x)].head(10)

6     Alpinultras. Sello, circuito y estudio médico para potenciar las travesías de ultrafondo de al… http://wp.me/p9GIl-b3O  vía @CarrerasMontana 
7     5 people unfollowed me today tracked by http://unfollowspy.com                                                                               
9     Wir wünschen euch von Herzen ein gesundes 2016! http://fb.me/7SfcGRpCP                                                                       
17    Aktuellste Stellenangebote http://tinyurl.com/bugpgs8                                                                                        
23    Naa geierst Du wieder und willst wichsen..Na gut aber Du musst 2 mal spritzen.Du musst… http://dlvr.it/D8jjgL  pic.twitter.com/LBV2QcdDIF    
25    AZEALIA BANKS - 212 FT. LAZY JAY http://fb.me/3GqunbPRp                                                                                      
26    Alles Gute fürs Neue Jahr! Sehen Sie eine Videobotschaft des CEO #Bringhen Group: https://vimeo.com/148510

Looking more intently at the tweets containing links, we can make the assumption that the relationship between URLs and spam is a general rule (at any time of the year).



Preform a vastly simplified version of the dictionary matching we will preform to get relevant tweets and analyze the results.

Here we look at the occurence of 'suicide' in the tweet:

In [13]:
pd.set_option('display.max_colwidth', 100)
twitter_df[twitter_df['main'].map(lambda x: 'suicide' in x) ]['main']

1731     Suicide bombing kills 1 at Kabul restaurant - http://news.bridgeward.com/suicide-bombing-kills-1...
3624                                              IS suicide attack planned in Munich http://bit.ly/1Owgpls 
5228     Afghanistan: attentat suicide dans un restaurant français de Kaboul - Europe1 http://dlvr.it/D8q...
6896     Ian Murdock's last night alive #debian http://sanfrancisco.cbslocal.com/2015/12/31/prominent-pro...
9479     Ipad Music Making Daily is out! http://paper.li/suicidesurfer77/1315555100?edition_id=bb78ee60-b...
10145     L'Etat islamique soupçonné d'avoir planifié un attentat suicide en Allemagne http://rss.ch/662786 
10446                           Afghanistan: attentat suicide dans le centre de Kaboul http://rss.ch/662843 
11096    Kaboul : les talibans revendiquent l'attentat suicide de ce 1er janvier contre un restaurant fra...
11377     L'Etat islamique soupçonné d'avoir planifié un attentat suicide en Allemagne http://rss.ch/662783 
14771              

By performing a simplistic dictionary matching using the occurences of the word *'suicide'* in our tweet set, we see that a lot of these tweets contain news. This further comforts us in our choice to remove tweets with URLs in order to get better results.

In [14]:
twitter_df[twitter_df['main'].map(lambda x: 'therapie' in x) ]['main']

11567    #Arbeit #Jobs #CH Chefarzt Psychiatrie/Psychotherapie 100 (w/m): Herisau, Appenzell Ausserrhoden...
12508    Aktuelle Firmensuche "Physiotherapie Praxis Bücheli" Ostermundigen (BE) #Gesundheitsberatung #Su...
13610    Natürlich Gesund: Mistel und Myrrhe in der Phytotherapie http://pure-natur.blogspot.ch/2014/06/t...
13720    Aktuelle Suchabfrage "Kunsttherapie" auf @Help_ch #Kunsttherapie #Suchportal #Schweiz #Suche htt...
Name: main, dtype: object

Looking at the word *'therapy'* (in German), we can confirm once again the issue there is with URLs.

In [15]:
twitter_df[twitter_df['main'].map(lambda x: 'RT ' in x) ]['main'].head()

124     RT @TimeOutSwiss : Discover some of the best ski resorts in #Switzerland #Swissalps #SwissSki #s...
3472    RT @SkafarPierre: General differences in use of social media for health care #hcsm #digitial #so...
3697                           RT @Boehringer: We wish you a happy new year! #Hcsm #hcsmeu #socmed #newyear
3751    RT @Paul_Sonnier: Tech That Will Change Your Life in 2016 http://stfi.re/xzzdya  #IoT #Whealth #...
3786    touche.... https://twitter.com/JamesRobertWebb/status/682951862929080322 … And #SometimesWhenIFe...
Name: main, dtype: object

Finally, we look at the retweets to get an idea on how useful they could be. From what we see, and what others have seen [8], they are not.

Using this analysis, we can apply this gained knowledge to start cleaning our data.

## 2. Dataset Cleaning

Now that we have done some data exploration, we have to clean our dataset to be able to use it correctly for our research.

First, we use Pandas locally on a small subset of the tweets to explore different cleaning methods and make sure the functions we chose work as expected. After proving that our concept works, we use Spark to scale up our operations and be able to perform them on the 320 files provided on the cluster.

_Note: The Spark version of this code can be found in the [run.py](run.py) file._

### 2.1 Unnesting the JSON

As mentioned in _Part 1_, the provided dataset uses a nested json format. Thus, to be able to work with it, we need to unnest it using the _normalize_ function.

_Note: we normalize the **data** DF as we use it in **Part 1** to import our datasets._

In [16]:
cleaned = json_normalize(data)
cleaned.columns = [column.replace('_source.','') for column in cleaned.columns]

### 2.2 Column Selection

Once we have a functional Dataframe, we choose the useful columns determined above.

In [17]:
cleaned = cleaned[['main', 'published', 'source_spam_probability', 'source_location', 'tags', 'lang', 'sentiment',
                   'author_gender', 'source_followers', 'source_following']]

As mentionned in _Part 1_, we chose this columns for the possible insights they can give us. Some of the question we want to answer in our research are:
- _Are men or women more prone to depression?_ 
- _Is someone with more followers happy?_

### 2.3 Language Filtering

The dataset provided contains tweets in a lot of languages. However, as our project is focused on the Swiss Population, we only keep tweets written in one of the official Swiss languages (i.e. French, German and Italian).

However, we had 2 hurdles (pushing us to do slight changes):
- Nobody in the group is Italian-Speaking
- Most of the tweets are written in English _(this is due to the large English-Speaking community in Switzerland but also due to the fact English is a _de-facto_ choice when communicating through the Internet)_

Moreover, we should not forget the existence of Swiss-German dialects which also complicates the task at hand.

To work our way through these problems, we decided to keep 3 languages: English, French and German. Note that we mean "German" in an extended manner as we also include Swiss-German dialects in the lot (this will be taken into account when creating our dictionaries as some words slightly change between the two).

In [18]:
lang_mask = ~cleaned.lang.isin(['de', 'en', 'fr'])
cleaned.drop(cleaned[lang_mask].index, inplace=True)
cleaned.reset_index(drop=True, inplace=True)

### 2.4 Sentiment Analysis

As we use a dataset with sentiment labeling, we use this opportunity to drop all tweets marked as **POSITIVE** given the nature of our subject (we are looking for signs of mental distress). 

In [19]:
sent_mask = (cleaned['sentiment'] == 'POSITIVE')
cleaned.drop(cleaned[sent_mask].index, inplace=True)
cleaned.reset_index(drop=True, inplace=True)

As this labeling is largely based on emojis, we expect some false positives in the **POSITIVE** category. However, it is too difficult to consider the sarcastic use of the _:)_ emoji (text analysis is not refined enough yet to detect such subtleties). Thus, we knowingly choose this method as we are only interested in having an overview of tweets showcasing signs of mental illnesses and not listing all of them.

### 2.5 Spam Removal

Spam is not relevant to any of our objectives. Thus, we decide to drop all tweets having a probability of being a spam greater value than 0.5 (given in the **source_spam_probability** field). Once again, we decide to trust the system that treated the data.

In [20]:
spam_mask = (cleaned['source_spam_probability'] >= 0.5)
cleaned.drop(cleaned[spam_mask].index, inplace=True)
cleaned.reset_index(drop=True, inplace=True)

_Note: our choice to have a 50% threshold follows the maximum likelihood rule and was considered to be a sufficiently robust value._

### 2.6 Time Format Encoding

To be able to use the time in our analysis (especially to find seasonal patterns in depression), we need the dates to be properly formatted.

In [21]:
cleaned['published'] = pd.to_datetime(cleaned['published'])
cleaned['published'].head()

0   2016-01-01 00:30:04
1   2016-01-01 12:12:33
2   2016-01-01 19:52:15
3   2016-01-01 11:58:03
4   2016-01-01 06:17:28
Name: published, dtype: datetime64[ns]

### 2.7 Text Treatment

Now that we only have relevant tweets (following a mild cleaning), we need to work on the main subject of our analysis: the content of the tweets itself. This is necessary to ease the _Part 3_ of our research allowing us to process the text and find patterns: Natural Language Processing.

The first step of our treatment is to transform all the caracters to lowercase to be able to compare tweets easily without being bothered with case-sensitive searches. Following the same idea, we get rid of all the special characters complicating our task.

In [22]:
cleaned['main'] = cleaned['main'].astype(str).str.lower().\
                    apply(lambda tweet: unicodedata.normalize('NFD', tweet).\
                    encode('ascii', 'ignore').decode('utf-8'))
cleaned['main'].head()

0                                                 en esperant que 2016 soit meilleur que 2015 quand meme
1                                               nice bmw ...he kills ant s!!! pic.twitter.com/5zt5v1mljk
2    @madmenna ich hab nichts dagegen, wenn mir andere beim saufen zuschauen, solange sie pro minute ...
3     happy new jear!!!! i am back from 2 weeks cuba! let s go into the 2016! pic.twitter.com/bu0lbpfql3
4    @megadriver16 bonne annee a toi je prends beaucoup de plaisir a suivre tes videos je suis un gra...
Name: main, dtype: object

As we have seen in _Part 1_, URLs are highly linked to spam (except for URLs linking to pictures). As we image processing is not in the scope of this project, we remove all "pic.twitter" URLS. After that, we take out all tweets containing a URL format. We also remove retweets, as they would 

In [23]:
url_mask = cleaned['main'].str.contains("www\S+") | cleaned['main'].str.contains("http\S+")
cleaned['main'] = cleaned['main'].str.replace("pic.twitter\S+", '')
cleaned.drop(cleaned[url_mask].index, inplace=True)
cleaned.reset_index(drop=True, inplace=True)
cleaned['main'].head()

0                                                 en esperant que 2016 soit meilleur que 2015 quand meme
1                                                                         nice bmw ...he kills ant s!!! 
2    @madmenna ich hab nichts dagegen, wenn mir andere beim saufen zuschauen, solange sie pro minute ...
3                               happy new jear!!!! i am back from 2 weeks cuba! let s go into the 2016! 
4    @megadriver16 bonne annee a toi je prends beaucoup de plaisir a suivre tes videos je suis un gra...
Name: main, dtype: object

We also remove mentions to remove the usernames, which are meanling less and should not be considered:

In [24]:
cleaned['main'] = cleaned['main'].str.replace("@\S+", '')

Finally, we remove all non-alphanumeric characters, as they are will not give more information and might get in the way of the text analysis.

In [25]:
cleaned['main'] = cleaned['main'].str.replace(r'[^\w\s]', '')
cleaned.main.head()

0                                                 en esperant que 2016 soit meilleur que 2015 quand meme
1                                                                               nice bmw he kills ant s 
2          ich hab nichts dagegen wenn mir andere beim saufen zuschauen solange sie pro minute 5 zahlen 
3                                     happy new jear i am back from 2 weeks cuba let s go into the 2016 
4     bonne annee a toi je prends beaucoup de plaisir a suivre tes videos je suis un grand fande tour...
Name: main, dtype: object

After performing all these cleaning steps, the size of the set is significantly reduced (the new file only weights 2.4MB instead of the 55MB of the original file).

## 3. NLP methods

Using Natural Language Processing (NLP) is necessary given the nature of our dataset: tweets. Following the steps used in previous courses and in the papers we read, we came up with the following pipeline in order to process the tweets (and dictionary). Unlike the previous part, we only used local functions instead of a scaled up version using Spark.

_Note: we use nltk, a goto python NLP library which was very interesting for us as it offered operations in the various languages we are working on._

### 3.1. Tokenizing

The first important step in NLP is to divide the words of every tweet in a table to be able to easily treat them. It is easily done using the _'split'_ method provided in Python. 

In [26]:
cleaned['tweets'] = cleaned['main'] #keep tweets for later
cleaned['main'] = cleaned['main'].str.split()
cleaned.head()

,main,published,source_spam_probability,source_location,tags,lang,sentiment,author_gender,source_followers,source_following,tweets
0,"[en, esperant, que, 2016, soit, meilleur, que, 2015, quand, meme]",2016-01-01 00:30:04,0.0,Saint-Maurice,NaN,fr,NEUTRAL,FEMALE,80,140,en esperant que 2016 soit meilleur que 2015 quand meme
1,"[nice, bmw, he, kills, ant, s]",2016-01-01 12:12:33,0.0,Hinwil,NaN,en,NEUTRAL,UNKNOWN,79,196,nice bmw he kills ant s
2,"[ich, hab, nichts, dagegen, wenn, mir, andere, beim, saufen, zuschauen, solange, sie, pro, minut...",2016-01-01 19:52:15,0.0,Schenkon,NaN,de,NEUTRAL,MALE,479,783,ich hab nichts dagegen wenn mir andere beim saufen zuschauen solange sie pro minute 5 zahlen
3,"[happy, new, jear, i, am, back, from, 2, weeks, cuba, let, s, go, into, the, 2016]",2016-01-01 11:58:03,0.0,Hinwil,NaN,en,NEUTRAL,UNKNOWN,79,196,happy new jear i am back from 2 weeks cuba let s go into the 2016
4,"[bonne, annee, a, toi, je, prends, beaucoup, de, plaisir, a, suivre, tes, videos, je, suis, un, ...",2016-01-01 06:17:28,0.0,Bâle,NaN,fr,NEUTRAL,UNKNOWN,75,984,bonne annee a toi je prends beaucoup de plaisir a suivre tes videos je suis un grand fande tour...


### 3.2 Stop words Removal

To have relevant tweets, it is necessary to remove useless words which would interfere with our analysis. We assumed stop words were the only important words to remove (as we already dealt with special characters and urls).

In [27]:
def remove_stops(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    lang_set = stopwords.words(language)
        
    cleaned.loc[cleaned['lang'] == lang, 'main'] = cleaned.loc[cleaned['lang'] == lang, 'main'].\
        apply(lambda tweet: [word for word in tweet if word not in lang_set])

In [28]:
remove_stops('english')
remove_stops('french')
remove_stops('german')
cleaned.head()

,main,published,source_spam_probability,source_location,tags,lang,sentiment,author_gender,source_followers,source_following,tweets
0,"[esperant, 2016, meilleur, 2015, quand, meme]",2016-01-01 00:30:04,0.0,Saint-Maurice,NaN,fr,NEUTRAL,FEMALE,80,140,en esperant que 2016 soit meilleur que 2015 quand meme
1,"[nice, bmw, kills, ant]",2016-01-01 12:12:33,0.0,Hinwil,NaN,en,NEUTRAL,UNKNOWN,79,196,nice bmw he kills ant s
2,"[dagegen, beim, saufen, zuschauen, solange, pro, minute, 5, zahlen]",2016-01-01 19:52:15,0.0,Schenkon,NaN,de,NEUTRAL,MALE,479,783,ich hab nichts dagegen wenn mir andere beim saufen zuschauen solange sie pro minute 5 zahlen
3,"[happy, new, jear, back, 2, weeks, cuba, let, go, 2016]",2016-01-01 11:58:03,0.0,Hinwil,NaN,en,NEUTRAL,UNKNOWN,79,196,happy new jear i am back from 2 weeks cuba let s go into the 2016
4,"[bonne, annee, a, prends, beaucoup, plaisir, a, suivre, videos, grand, fande, tour, france, merc...",2016-01-01 06:17:28,0.0,Bâle,NaN,fr,NEUTRAL,UNKNOWN,75,984,bonne annee a toi je prends beaucoup de plaisir a suivre tes videos je suis un grand fande tour...


### 3.3 Stemming

The final step of our NLP treatement pipeline is stemming. The idea is that words appearing in multiple forms (such as have, having, had, …) should only be considered once using their radical (e.g. "hav").

In [29]:
def stem_words(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    stemmer = SnowballStemmer(language)
        
    cleaned.loc[cleaned['lang'] == lang, 'main'] = cleaned.loc[cleaned['lang'] == lang, 'main'].\
        apply(lambda tweet: [stemmer.stem(word) for word in tweet])

In [30]:
stem_words('english')
stem_words('french')
stem_words('german')
cleaned.head()

,main,published,source_spam_probability,source_location,tags,lang,sentiment,author_gender,source_followers,source_following,tweets
0,"[esper, 2016, meilleur, 2015, quand, mem]",2016-01-01 00:30:04,0.0,Saint-Maurice,NaN,fr,NEUTRAL,FEMALE,80,140,en esperant que 2016 soit meilleur que 2015 quand meme
1,"[nice, bmw, kill, ant]",2016-01-01 12:12:33,0.0,Hinwil,NaN,en,NEUTRAL,UNKNOWN,79,196,nice bmw he kills ant s
2,"[dageg, beim, sauf, zuschau, solang, pro, minut, 5, zahl]",2016-01-01 19:52:15,0.0,Schenkon,NaN,de,NEUTRAL,MALE,479,783,ich hab nichts dagegen wenn mir andere beim saufen zuschauen solange sie pro minute 5 zahlen
3,"[happi, new, jear, back, 2, week, cuba, let, go, 2016]",2016-01-01 11:58:03,0.0,Hinwil,NaN,en,NEUTRAL,UNKNOWN,79,196,happy new jear i am back from 2 weeks cuba let s go into the 2016
4,"[bon, anne, a, prend, beaucoup, plais, a, suivr, videos, grand, fand, tour, franc, merc, continu...",2016-01-01 06:17:28,0.0,Bâle,NaN,fr,NEUTRAL,UNKNOWN,75,984,bonne annee a toi je prends beaucoup de plaisir a suivre tes videos je suis un grand fande tour...


### 3.4 Dictionary processing

As we applied all of these methods to our dataset, it is only natural that we do the same for the dictionary we will use. The first step of this part explains how we built the dictionary (and cleaned it as we did before) while the second part focuses on applying the previous NLP methods to the dictionary.

####  Building the dictionary

The first step of building our dictionary was doing research on previous dictionaries that were used for similar problems, such as can be seen in [2], [3]. Starting from this, we built our own dictionary by expanding the examples (as we expanded our subject to take into account multiple mental disorders instead of simply determining tweets of users diagnosed with clinical illnesses). As we treat tweets in multiple languages, we also translated all the terms and tried adding words that specifically target mood (or eating) disorders in french and german.

_Note: you can take a look at this dictionary by clicking on the 4th reference (at the end of the file)._

In [31]:
DICT_PATH = "dictionary.csv"
dictionaries = pd.read_csv(DICT_PATH)
dictionaries.head()

,english,french,german,swiss_german
0,abusive,abusif,missbrauch,NaN
1,addict,accro,sucht,NaN
2,afraid,peur,angst,angscht
3,agoraphobe,agoraphobe,NaN,NaN
4,agoraphobia,agoraphobie,NaN,NaN


In [32]:
en_dict = dictionaries['english'].dropna()
fr_dict = dictionaries['french'].dropna()
de_dict = pd.concat([dictionaries['german'].dropna(), dictionaries['swiss_german'].dropna()])

After fetching our 3 datasets, we clean them as we did for the tweets in _Part 2_. However, as we created them ourselves, we only have to lowercase them and treat the special characters. 

In [33]:
def dict_cleaning(lang):
    lang_dict = eval(lang + '_dict')
    lang_dict = lang_dict.astype(str).str.lower().\
                        apply(lambda expression: unicodedata.normalize('NFD', expression).\
                        encode('ascii', 'ignore').decode('utf-8'))

In [34]:
dict_cleaning('en')
dict_cleaning('fr')
dict_cleaning('de')

#### Processing the dictionary

Once we have our (clean) data, we simply run all the previous NLP methods on our dictionaries. 

_Note: As all methods were explained above, we will not dwell on each step._

In [35]:
#Tokenizing
en_dict = en_dict.str.split()
fr_dict = fr_dict.str.split()
de_dict = de_dict.str.split()

In [36]:
def dict_remove_stops(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    lang_dict = eval(lang + '_dict')
    lang_set = stopwords.words(language)
        
    lang_dict = lang_dict.apply(lambda expression: [word for word in expression if word not in lang_set])

In [37]:
#Removing stop words
dict_remove_stops('english')
dict_remove_stops('french')
dict_remove_stops('german')

In [38]:
def dict_stem_words(language):
    lang = language[:2]
    if language == 'german':
        lang = 'de'
    
    lang_dict = eval(lang + '_dict')
    stemmer = SnowballStemmer(language)
        
    lang_dict = lang_dict.apply(lambda expression: [stemmer.stem(word) for word in expression])

In [39]:
#Stemming the words
dict_stem_words('english')
dict_stem_words('french')
dict_stem_words('german')

To show the soundness of our method, we display the head of each dictionary:

In [40]:
en_dict.head()

0        [abusive]
1         [addict]
2         [afraid]
3     [agoraphobe]
4    [agoraphobia]
Name: english, dtype: object

In [41]:
fr_dict.head()

0         [abusif]
1          [accro]
2           [peur]
3     [agoraphobe]
4    [agoraphobie]
Name: french, dtype: object

In [42]:
de_dict.head()

0      [missbrauch]
1           [sucht]
2           [angst]
5    [alcoholismus]
6           [alein]
dtype: object

### 3.5 Processing the Data

In this part, we cross the dataset with our dictionaries to retrieve the tweets exhibiting mental distress. This is the first (less naive) step of our analysis before running the Machine Learning algorithms on our data.

We first transfrom the lists back to string sentences, for efficacy and code readability:

In [43]:
cleaned.main = cleaned.main.map(lambda x:  ' '.join(x))
de_dict = de_dict.map(lambda x:  ' '.join(x))
en_dict = en_dict.map(lambda x:  ' '.join(x))
fr_dict = fr_dict.map(lambda x:  ' '.join(x))

We now check if we can find a dict entry for each tweet

In [44]:
def check_dict(tweet, dict_):
    """checks if dict entry matches tweet"""
    match = [ w for w in dict_ if w in tweet] #find matching for each entry
    return len(match) > 0 #at least one match

This gives us the following results for each language.


For english:

In [45]:
english_filtered = cleaned[cleaned.lang == 'en']['tweets']\
[cleaned[cleaned.lang == 'en']['main'].map(lambda x: check_dict(x, en_dict))]
english_filtered.head()

210     i feel sad that they have to hide and probably wont ever be able to go out and have fun like a ...
279     it become a scandal like if dating someone is wrongand they may loose fansor getting hate from ...
314                                                 i hope he will take care of my girl dont hurt her huh 
316                                   i hope people will react positively but im afraid about his fangirls
349    but hanis dating news makes me feel sad because its not normal that idols have to hide their rel...
Name: tweets, dtype: object

In [46]:
english_filtered.count()

106

We can clearly see sadness an lonelyness or fear in this tweets, but some of them do not actualy express distress.
We also note that the number of tweets has greatly been reduces through this.

Now we look at the french tweets and again find similar results:

In [47]:
french_filtered = cleaned[cleaned.lang == 'fr']['tweets']\
[cleaned[cleaned.lang == 'fr']['main'].map(lambda x: check_dict(x, fr_dict))]
french_filtered.head()

310     je te comprends tellement cest vraiment se foutre de la gueule des lecteurs milady sont les roi...
330                                                                                suis je le seul debout 
343                         je crois je vais allez regarder les feu dartifices toute seule au bord de leau
411                jai ajoute 60 livres dans ma boutique priceminister cest ezila4 fouillez il y a de tout
588               trouve le temps de passer sur le mumble et le voit avec personnes de present tristesse d
Name: tweets, dtype: object

In [48]:
french_filtered.count()

96

Finally, for german we get the following results:

In [49]:
german_filtered = cleaned[cleaned.lang == 'de']['tweets']\
[cleaned[cleaned.lang == 'de']['main'].map(lambda x: check_dict(x, de_dict))]
german_filtered.head()

65     rtl bringt das kunststuck fertig eine klassische pleitenpechundpannensendung in den sand zu setz...
426                                                            das war er nun der schlechteste tatort ever
458    angekundigte katastrophen bleiben in der regel aus wem nutzt die angstmache eigentlich mehr den ...
548                        nur so wenn du schlaftst die zeit auch vergeht und das meist sogar schmerzfrei 
558                                                         nei nei keine angst ich reise nicht mehr gerne
Name: tweets, dtype: object

In [50]:
german_filtered.count()

61

Not only do all of this tweets reference the person, but they also talk about sadness or dissapointment.
The remaining number is quite small, compared to the initial 4000 german tweets in the dataset, but it is a number we migth expect.

Still those results could be better, which is why we decide to introduce a step 4 into our analysis:

## 4. ML (to be done in milestone 3)

The final part of our data processing involves training a supervised Machine Learning classifier to get better results, as missclasifications are still somewhat frequent. Similar projects routinely use this [2] [3] [8].

All these methods are implemented in scikit-learn and have an equivalent in Spark in case our computers cannot handle the size of the data, allowing us to perform these steps on the cluster.

In order to do this, we go through the following steps:

### 4.1. Labeling the tweets

We need to label a subset of tweets as 'mentaly distressed' or not. Ideally around 6000 per language [7], but due to limited time and manpower, we will limit ourself to 1000 tweets each.

### 4.2 Construcing features TF-IDF

Once we have the labeled set, we transfrom it in order to have features to use for the ML algorithm.

### 4.3 Train SVM classifier

According to [7], SVM along with TF-IDF tends to preform very well, especially with limited training set size (which is ideal in our case). We perform a binary classification on the labeled dataset.

### 4.4 Relabel the training set

Using the previously attained classifier, we relabel the set and discard tweets that are labeled as 'undistressed'.

## 5. Final Data Analysis

In the final analysis, we try to analyze and visualize the results in order to answer our research questions.

### 5.1 General analysis

We compare the set found using Machine Learning to the general cleaned set and the overall set.

### 5.2 Finding related indicatior of mental health issues using LDA

As discussed in [8], LDA one of the best ways to find relevant related words in tweets. Again, we plan on using the implemented scikit-learn version to preform this.

### 5.3 Lookint at gender differences

### 5.4 Tweet frequency and type over time

### 5.5 Comparing our results to census data

This will be approximative since we could not find a dataset containing this information.

### 5.6 Comparing the languages

## 6. Conclusion

# References and Bibliography

[1] [Example dataprocessing pipeline](http://nbviewer.jupyter.org/gist/mizvol/eb24770ac3d5d598463f972e2a669f03)

[2] [Dissertation containing a first dict](https://www.rand.org/content/dam/rand/pubs/rgs_dissertations/RGSD300/RGSD391/RAND_RGSD391.pdf)

[3] [Second thesis containing dict](https://getd.libs.uga.edu/pdfs/kale_sayali_s_201512_ms.pdf)

[4] [Our own dictionary](https://docs.google.com/spreadsheets/d/1WwI9crZk36pcTOQ1g_5dumMd11OlkpFRNHsEvpkwLMk/edit?usp=sharing)

[5] [Stemming with Spark](https://github.com/master/spark-stemming)

[6] [ML methods in Spark](https://spark.apache.org/docs/2.1.0/ml-features.html)

[8] [Public health paper using LDA](https://www.aaai.org/ocs/index.php/ICWSM/ICWSM11/paper/viewFile/2880/3264)

[7] [Best ways to do Text Classification](http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/)